In [0]:
# !pip install tensorflow-hub
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow_hub as hub
import tensorflow as tf

In [0]:
import os
os.environ['TFHUB_DOWNLOAD_PROGRESS'] = "1"

In [0]:
!pip install zh_dataset_inews
!pip install tqdm
from tqdm import tqdm

In [0]:
from zh_dataset_inews import title_train, content_train, label_train
from zh_dataset_inews import title_dev, content_dev, label_dev
from zh_dataset_inews import title_test, content_test, label_test
from sklearn.metrics import accuracy_score

In [0]:
def compose_data(title, content, label):
  x = tf.constant([
      [(t + '[SEP]' + c)]
      for t, c in zip(title, content)
  ])
  y = tf.constant(tf.keras.utils.to_categorical(label, 3))
  return x, y

In [0]:
x_train, y_train = compose_data(title_train, content_train, label_train)
x_dev, y_dev = compose_data(title_dev, content_dev, label_dev)
x_test, y_test = compose_data(title_test, content_test, label_test)

In [0]:
model = tf.keras.Sequential([
  hub.KerasLayer(
    'https://github.com/qhduan/bert-model/releases/download/1.0/roberta_wwm_zh_pool.tar.gz',
    trainable=True),
  tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.optimizers.Adam(5e-6),
  metrics=['accuracy']
)

model.fit(x_train, y_train, validation_data=(x_dev, y_dev), epochs=3, batch_size=4)

Train on 5355 samples, validate on 999 samples
Epoch 1/3
5355/5355 [==============================] - 746s 139ms/sample - loss: 0.4902 - accuracy: 0.8058 - val_loss: 0.3580 - val_accuracy: 0.8448
Epoch 2/3
5355/5355 [==============================] - 751s 140ms/sample - loss: 0.2739 - accuracy: 0.8913 - val_loss: 0.4624 - val_accuracy: 0.8298
Epoch 3/3
5355/5355 [==============================] - 750s 140ms/sample - loss: 0.1664 - accuracy: 0.9382 - val_loss: 0.4284 - val_accuracy: 0.8488


In [0]:
logits_test = model.predict(x_test, verbose=1)
pred_test = logits_test.argmax(-1).tolist()

999/999 [==============================] - 47s 47ms/sample


In [0]:
print(accuracy_score(label_test, pred_test))

0.8338338338338338
